#### Import relevant libraries and read the data

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import preprocessing

# Set to raise an error for chained assignment
pd.set_option('mode.chained_assignment', 'raise')

# Read the data as pandas DataFrame
data = pd.read_csv('../../data/prostate.csv')

#### Split the data into train/test data

In [2]:
# Y: response variables, train_label: checks whether it is for training or not
Y = data.pop('lpsa')
train_label = data.pop('train')

# Divide the data into train/test
train_X = data.loc[train_label == 'T', :].copy()
train_Y = Y[train_label == 'T'].copy()
test_X = data.loc[train_label == 'F', :].copy()
test_Y = Y[train_label == 'F'].copy()
'''
# Divide into train and test input Randomly!
train_X = data.sample(n=67, random_state=5, axis=0)
train_Y = Y[train_X.index].copy()

test_X = data.drop(train_X.index)
test_Y = Y[test_X.index].copy()
'''

# Feature names
features = data.columns.values

#### Normalize the training predictor values and fit the linear mode into the data

In [3]:
# Normalize the input
scaler = preprocessing.StandardScaler().fit(train_X)
train_X.loc[:, features] = scaler.transform(train_X)

# add 1's for the intercept
train_X = sm.add_constant(train_X)
train_X.rename(columns={'const': 'Intercept'}, inplace=True)

# Linear Regression Result
ols = sm.OLS(train_Y, train_X)
ols_res = ols.fit()
print(ols_res.summary())

                            OLS Regression Results                            
Dep. Variable:                   lpsa   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                     16.47
Date:                Tue, 02 Feb 2021   Prob (F-statistic):           2.04e-12
Time:                        22:50:16   Log-Likelihood:                -67.505
No. Observations:                  67   AIC:                             153.0
Df Residuals:                      58   BIC:                             172.9
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.4523      0.087     28.182      0.0

#### Compared to the base model (average of training response), the linear model reduces 50.67% of test Mean Squared Error (MSE)

In [4]:
# Scale test input with mean, and variance from train input
test_X.loc[:, features] = scaler.transform(test_X)
test_X = sm.add_constant(test_X)
test_X.rename(columns={'const': 'Intercept'}, inplace=True)

# Predicted values and MSE for the test input
predicted_Y = ols_res.predict(test_X)
test_MSE = sum((predicted_Y - test_Y)**2) / test_Y.shape[0]

# As a base error, we use the average of the train_Y
base_MSE = sum((np.mean(train_Y) - test_Y)**2) / test_Y.shape[0]

print(f'MSE from Linear Model: {test_MSE:.4f}')
print(f'MSE from base Model: {base_MSE:.4f}')
print(f'MSE decreased by {(base_MSE - test_MSE)/base_MSE*100:.2f}% with the linear model')

MSE from Linear Model: 0.5213
MSE from base Model: 1.0567
MSE decreased by 50.67% with the linear model


#### We have followed the ESL so far, but we may have a better result if we introduce some categorical features (age, svi, gleason)

#### 5 additional binary variables are added + 1 svi (un-normalized)
- 'age_lo', 'age_mid', 'age_hi'
- 'gleason_6', 'gleason_7'

In [5]:
# We make dummy variables to handle categorical features
# age: low (<=55),
#      mid (55< <=65)
#      hi  (>65)

# svi: 0 vs 1

# gleason: 6 vs 7 vs {8, 9}

new_data = data.copy()

# age dummies
new_data['age_lo'] = (data.loc[:, 'age'] <= 55)
new_data['age_mid'] = (55 < data.loc[:, 'age']) & (data.loc[:, 'age'] <= 65)
new_data['age_hi'] = (data.loc[:, 'age'] > 65)

new_data.loc[:, 'age_lo'] = (new_data.loc[:, 'age_lo']).astype(int)
new_data.loc[:, 'age_mid'] = (new_data.loc[:, 'age_mid']).astype(int)
new_data.loc[:, 'age_hi'] = (new_data.loc[:, 'age_hi']).astype(int)
new_data.pop('age')

# svi -> just leave it without normalizing

# gleason dummies
new_data['gleason_6'] = (data.loc[:, 'gleason'] == 6)
new_data['gleason_7'] = (data.loc[:, 'gleason'] == 7)

new_data.loc[:, 'gleason_6'] = (new_data.loc[:, 'gleason_6']).astype(int)
new_data.loc[:, 'gleason_7'] = (new_data.loc[:, 'gleason_7']).astype(int)
new_data.pop('gleason');

#### The upper part follows the same split of train/test data as in ESL, whereas the lower part splits the data randomly

In [6]:

# Divide into train and test input (Same as in ESL)
new_train_X = new_data.loc[train_label == 'T', :].copy()
new_train_Y = Y[train_label == 'T'].copy()

new_test_X = new_data.loc[train_label == 'F', :].copy()
new_test_Y = Y[train_label == 'F'].copy()
'''
# Divide into train and test input Randomly!
new_train_X = new_data.sample(n=67, random_state=5, axis=0)
new_train_Y = Y[new_train_X.index].copy()

new_test_X = new_data.drop(new_train_X.index)
new_test_Y = Y[new_test_X.index].copy()
'''

'\n# Divide into train and test input Randomly!\nnew_train_X = new_data.sample(n=67, random_state=5, axis=0)\nnew_train_Y = Y[new_train_X.index].copy()\n\nnew_test_X = new_data.drop(new_train_X.index)\nnew_test_Y = Y[new_test_X.index].copy()\n'

#### Normalize only the qunatitative variables, and fit the model

In [7]:
# Normalize except age, svi, gleason variables
quant_features = ['lcavol', 'lweight', 'lbph', 'lcp', 'pgg45']
normalizer = preprocessing.StandardScaler().fit(new_train_X.loc[:, quant_features])
new_train_X.loc[:, quant_features] = normalizer.transform(new_train_X.loc[:, quant_features])

# add 1's for the intercept
new_train_X = sm.add_constant(new_train_X)
new_train_X.rename(columns={'const': 'Intercept'}, inplace=True)

# Linear Regression Result
new_ols = sm.OLS(new_train_Y, new_train_X)
new_ols_res = new_ols.fit()
print(new_ols_res.summary())

                            OLS Regression Results                            
Dep. Variable:                   lpsa   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     12.96
Date:                Tue, 02 Feb 2021   Prob (F-statistic):           2.43e-11
Time:                        22:50:17   Log-Likelihood:                -67.070
No. Observations:                  67   AIC:                             156.1
Df Residuals:                      56   BIC:                             180.4
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.6013      0.331      4.843      0.0

#### Different split gave lower MSE, but to justfy that introducing the categorical variables is the better model, you need to repeat random split and average the result

In [8]:
# Scale test input with mean, and variance from train input
new_test_X.loc[:, quant_features] = normalizer.transform(new_test_X.loc[:, quant_features])
new_test_X = sm.add_constant(new_test_X)
new_test_X.rename(columns={'const': 'Intercept'}, inplace=True)

# Predicted values and MSE for the test input
new_predicted_Y = new_ols_res.predict(new_test_X)
new_test_MSE = sum((new_predicted_Y - new_test_Y)**2) / new_test_Y.shape[0]

# As a base error, we use the average of the training response
base_MSE = sum((np.mean(new_train_Y) - new_test_Y)**2) / new_test_Y.shape[0]

print(f'MSE from New Linear Model: {new_test_MSE:.4f}')
print(f'MSE from base Model: {base_MSE:.4f}')
print(f'MSE decreased by {(base_MSE - new_test_MSE)/base_MSE*100:.2f}% with the new linear model')

MSE from New Linear Model: 0.4692
MSE from base Model: 1.0567
MSE decreased by 55.60% with the new linear model
